In [1]:
import os, re, json, numpy as np, pandas as pd
from datetime import datetime
import lightgbm as lgb

In [17]:
df = pd.read_excel("../../../data/train_l2_1.xlsx")

In [23]:
pairs = df.copy()

In [30]:
drop_leak = {'label','student_group','hb_ok','cand_budget_ok','cand_margin','cand_budget_diff'}
keep_cols = [c for c in pairs.columns if c not in drop_leak]

diff_cols = []
for c in ['hk10','hk11','hk12','hl10','hl11','hl12']:
    sc, cc = f'student_{c}', f'cand_{c}'
    if sc in pairs.columns and cc in pairs.columns:
        pairs[f'diff_{c}'] = pd.to_numeric(pairs[cc], errors='coerce') - pd.to_numeric(pairs[sc], errors='coerce')
        diff_cols.append(f'diff_{c}')


num_cols = [c for c in [
    'student_diem_chuan','student_budget_max',
    'cand_diem_chuan_final','cand_hoc_phi','cand_y_base'
] if c in keep_cols]
num_cols += diff_cols

cat_cols = [c for c in [
    'student_cong_lap','student_tinh_tp','student_to_hop_mon','student_ten_ccta','student_diem_ccta','student_nhom_nganh',
    'cand_cong_lap','cand_tinh_tp','cand_to_hop_mon','cand_nhom_nganh','cand_ma_xet_tuyen','cand_is_base_row'
] if c in keep_cols]

for c in num_cols:
    pairs[c] = pd.to_numeric(pairs[c], errors='coerce')
for c in cat_cols:
    pairs[c] = pairs[c].astype('category')

In [31]:
rng = np.random.default_rng(42)
users = pairs['student_group'].unique()
rng.shuffle(users)
cut = int(0.8*len(users))
train_u, valid_u = set(users[:cut]), set(users[cut:])
train_df = pairs[pairs['student_group'].isin(train_u)].copy()
valid_df = pairs[pairs['student_group'].isin(valid_u)].copy()

X_train = train_df[num_cols + cat_cols]
y_train = train_df['label'].astype(int)
X_valid = valid_df[num_cols + cat_cols]
y_valid = valid_df['label'].astype(int)

In [32]:
X_train.columns

Index(['student_diem_chuan', 'student_budget_max', 'cand_diem_chuan_final',
       'cand_hoc_phi', 'cand_y_base', 'diff_hk10', 'diff_hk11', 'diff_hk12',
       'diff_hl10', 'diff_hl11', 'diff_hl12', 'student_cong_lap',
       'student_tinh_tp', 'student_to_hop_mon', 'student_ten_ccta',
       'student_diem_ccta', 'student_nhom_nganh', 'cand_cong_lap',
       'cand_tinh_tp', 'cand_to_hop_mon', 'cand_nhom_nganh',
       'cand_ma_xet_tuyen', 'cand_is_base_row'],
      dtype='object')

In [33]:
d_train = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_cols, free_raw_data=False)
d_valid = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_cols, reference=d_train, free_raw_data=False)

params = {
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'num_leaves': 63,
    'subsample': 0.9,
    'colsample_bytree': 0.8,
    'verbose': -1,
    'seed': 42,
}
evals_result = {}
clf = lgb.train(
    params,
    d_train,
    num_boost_round=600,
    valid_sets=[d_train, d_valid],
    valid_names=['train','valid'],
    callbacks=[lgb.record_evaluation(evals_result), lgb.log_evaluation(50), lgb.early_stopping(100)]
)


Training until validation scores don't improve for 100 rounds
[50]	train's auc: 0.999436	valid's auc: 0.999034
[100]	train's auc: 0.999887	valid's auc: 0.999693
[150]	train's auc: 0.999972	valid's auc: 0.999837
[200]	train's auc: 0.999993	valid's auc: 0.999882
[250]	train's auc: 0.999998	valid's auc: 0.999895
[300]	train's auc: 0.999999	valid's auc: 0.99991
[350]	train's auc: 1	valid's auc: 0.999917
[400]	train's auc: 1	valid's auc: 0.999919
[450]	train's auc: 1	valid's auc: 0.999918
[500]	train's auc: 1	valid's auc: 0.999921
[550]	train's auc: 1	valid's auc: 0.999921
[600]	train's auc: 1	valid's auc: 0.999923
Did not meet early stopping. Best iteration is:
[594]	train's auc: 1	valid's auc: 0.999924


In [34]:
from sklearn.metrics import roc_auc_score, precision_score, recall_score

p = clf.predict(X_valid, num_iteration=clf.best_iteration)
auc = roc_auc_score(y_valid, p)
pred = (p >= 0.5).astype(int)
prec = precision_score(y_valid, pred, zero_division=0)
rec  = recall_score(y_valid, pred, zero_division=0)
print({'AUC': auc, 'precision@0.5': prec, 'recall@0.5': rec})

{'AUC': 0.9999235723067971, 'precision@0.5': 0.997635262958759, 'recall@0.5': 0.9993051827053658}


In [35]:
l2_uni = pd.read_excel("../../../data/l2_uni_requirement.xlsx")

In [36]:
student_input = [
    { 
        "diem_chuan": 800.0, 
        "hoc_phi": 40000000, 
        "cong_lap": 1, 
        "tinh_tp": "TP. Hồ Chí Minh", 
        "to_hop_mon": "VNUHCM", 
        "ten_ccta": "0", 
        "diem_ccta": "0", 
        "nhom_nganh": 714.0, 
        "hk10": "2", 
        "hk11": "2", 
        "hk12": "2", 
        "hl10": "2", 
        "hl11": "2", 
        "hl12": "2", 
    },
    { 
        "diem_chuan": 24.0, 
        "hoc_phi": 40000000, 
        "cong_lap": 1, 
        "tinh_tp": "TP. Hồ Chí Minh", 
        "to_hop_mon": "A00", 
        "ten_ccta": "0", 
        "diem_ccta": "0", 
        "nhom_nganh": 748.0, 
        "hk10": "2", 
        "hk11": "2", 
        "hk12": "2", 
        "hl10": "2", 
        "hl11": "2", 
        "hl12": "2", 
    },
    { 
        "diem_chuan": 21.0, 
        "hoc_phi": 40000000, 
        "cong_lap": 1, 
        "tinh_tp": "TP. Hồ Chí Minh", 
        "to_hop_mon": "A01", 
        "ten_ccta": "0", 
        "diem_ccta": "0", 
        "nhom_nganh": 748.0, 
        "hk10": "2", 
        "hk11": "2", 
        "hk12": "2", 
        "hl10": "2", 
        "hl11": "2", 
        "hl12": "2", 
    }
]
student_df = pd.DataFrame(student_input)

In [41]:
import polars as pl
def test_to_X(student_info: pl.DataFrame, candidate_list: pl.DataFrame) -> pl.DataFrame:

    cand_tp  = student_info['tinh_tp'].unique().to_list()
    cand_thm = student_info['to_hop_mon'].unique().to_list()
    cand_cl  = student_info['cong_lap'].unique().to_list()
    cand_nn  = student_info['nhom_nganh'].unique().to_list()

    candidate_uni_pl = candidate_list.filter(
        pl.col('tinh_tp').is_in(cand_tp) &
        pl.col('to_hop_mon').is_in(cand_thm) &
        pl.col('cong_lap').is_in(cand_cl) &
        pl.col('nhom_nganh').is_in(cand_nn)
    ).clone()

    student_info_pd   = student_info.to_pandas()
    candidate_uni_pd  = candidate_uni_pl.to_pandas()

    test_df = filter_candidates_per_student(
        student_info_pd,
        candidate_uni_pd,
        stu_budget_col='hoc_phi',
        cand_tuition_col='hoc_phi'
    )
    return pl.DataFrame(test_df)

In [38]:
# object to str
l2_uni = l2_uni.astype(str)

In [39]:
student_df = pl.from_pandas(student_df)
l2_uni = pl.from_pandas(l2_uni).with_columns([
    pl.col("cong_lap").cast(pl.Int64),
    pl.col("tinh_tp").cast(pl.Utf8),
    pl.col("to_hop_mon").cast(pl.Utf8),
    pl.col("diem_chuan").cast(pl.Float64),
    pl.col("hoc_phi").cast(pl.Int64),
    pl.col("ten_ccta").cast(pl.Utf8),
    pl.col("diem_ccta").cast(pl.Utf8),
    pl.col("diem_quy_doi").cast(pl.Float64),
    pl.col("hk10").cast(pl.Int64),
    pl.col("hk11").cast(pl.Int64),
    pl.col("hk12").cast(pl.Int64),
    pl.col("hl10").cast(pl.Int64),
    pl.col("hl11").cast(pl.Int64),
    pl.col("hl12").cast(pl.Float64),
    pl.col("nhom_nganh").cast(pl.Int64),
    pl.col("ma_xet_tuyen").cast(pl.Utf8)
])

In [52]:
import numpy as np
import pandas as pd

def filter_candidates_per_student(
    student_df: pd.DataFrame,
    candidate_df: pd.DataFrame,
    neg_pos_ratio: int = 3,
    hard_filters=None,
    stu_budget_col: str = 'hoc_phi',
    cand_tuition_col: str = 'hoc_phi',
    default_budget: float = np.inf
) -> pd.DataFrame:
    """
    Build (user, candidate) pairs for LightGBM ranking.

    Assumes candidate_df already has 'diem_chuan_final' (true cutoff after CCTA fix).
    Uses student's budget = student_df[stu_budget_col] and candidate's tuition = candidate_df[cand_tuition_col].

    Adds features:
      - cand_margin      = student_diem_chuan - cand_diem_chuan_final
      - cand_budget_ok   = 1 if tuition <= budget
      - cand_budget_diff = budget - tuition
    """
    if hard_filters is None:
        # IMPORTANT: do NOT include any fee column here
        hard_filters = ['tinh_tp', 'to_hop_mon', 'cong_lap', 'nhom_nganh']

    if 'diem_chuan_final' not in candidate_df.columns:
        raise ValueError("candidate_df missing 'diem_chuan_final' (run CCTA normalization first)")

    # ----- copies & dtype normalization -----
    cand = candidate_df.copy()
    cand['hoc_phi'] = pd.to_numeric(cand[cand_tuition_col], errors='coerce') if cand_tuition_col in cand.columns else np.nan

    stu = student_df.copy()
    stu['budget_max'] = pd.to_numeric(stu[stu_budget_col], errors='coerce') if stu_budget_col in stu.columns else default_budget
    if 'diem_chuan' in stu.columns:
        stu['diem_chuan'] = pd.to_numeric(stu['diem_chuan'], errors='coerce').fillna(0)

    cand_records = cand.to_dict('records')  # each is a dict of scalars
    cand_cols = set(cand.columns)
    stu_cols  = set(stu.columns)

    rows = []
    for _, stu_row in stu.iterrows():
        budget = float(stu_row.get('budget_max', default_budget))
        if not np.isfinite(budget) or pd.isna(budget):
            budget = default_budget

        # ---- hard filters (no hoc_phi equality) ----
        filtered = cand_records
        for k in hard_filters:
            if (k in stu_cols) and (k in cand_cols):
                v = stu_row.get(k)
                filtered = [cr for cr in filtered if cr.get(k) == v]

        # ---- budget filter ----
        filtered = [cr for cr in filtered if pd.notna(cr.get('hoc_phi')) and float(cr['hoc_phi']) <= budget]
        np.random.shuffle(filtered)

        suitable, not_suitable = [], []
        stu_score = float(stu_row.get('diem_chuan', 0.0))

        for cand_row in filtered:
            cutoff_final = float(cand_row.get('diem_chuan_final', 0.0))
            margin = stu_score - cutoff_final

            tuition = float(cand_row.get('hoc_phi', np.nan))
            budget_ok  = int(pd.notna(tuition) and tuition <= budget)
            budget_diff = (budget - tuition) if (pd.notna(tuition) and np.isfinite(budget)) else np.nan

            # Optional: transcript constraints (compare scalar↔scalar only if both present)
            hb_cols = ['hk10','hk11','hk12','hl10','hl11','hl12']
            ok_hb = True
            for col in hb_cols:
                s_val = pd.to_numeric(stu_row.get(col, np.nan), errors='coerce')
                c_val = pd.to_numeric(cand_row.get(col, np.nan), errors='coerce')
                if pd.notna(s_val) and pd.notna(c_val) and s_val > c_val:
                    ok_hb = False
                    break

            # Final label
            is_match = (margin >= 0) and bool(budget_ok) and ok_hb

            row = {}
            row.update({f'student_{k}': v for k, v in stu_row.items()})
            row.update({f'cand_{k}':    v for k, v in cand_row.items()})
            row['cand_diem_chuan_final'] = cutoff_final
            row['cand_margin'] = margin
            row['cand_budget_ok'] = budget_ok
            row['cand_budget_diff'] = budget_diff
            row['label'] = int(is_match)

            (suitable if is_match else not_suitable).append(row)

        rows.extend(suitable)
        if not_suitable:
            k = min(len(suitable) * neg_pos_ratio, len(not_suitable))
            if k > 0:
                rows.extend(np.random.choice(not_suitable, size=k, replace=False).tolist())
    pairs = pd.DataFrame(rows)
    drop_leak = {'label','student_group','hb_ok','cand_budget_ok','cand_margin','cand_budget_diff'}
    keep_cols = [c for c in pairs.columns if c not in drop_leak]

    diff_cols = []
    for c in ['hk10','hk11','hk12','hl10','hl11','hl12']:
        sc, cc = f'student_{c}', f'cand_{c}'
        if sc in pairs.columns and cc in pairs.columns:
            pairs[f'diff_{c}'] = pd.to_numeric(pairs[cc], errors='coerce') - pd.to_numeric(pairs[sc], errors='coerce')
            diff_cols.append(f'diff_{c}')


    num_cols = [c for c in [
        'student_diem_chuan','student_budget_max',
        'cand_diem_chuan_final','cand_hoc_phi','cand_y_base'
    ] if c in keep_cols]
    num_cols += diff_cols

    cat_cols = [c for c in [
        'student_cong_lap','student_tinh_tp','student_to_hop_mon','student_ten_ccta','student_diem_ccta','student_nhom_nganh',
        'cand_cong_lap','cand_tinh_tp','cand_to_hop_mon','cand_nhom_nganh','cand_ma_xet_tuyen','cand_is_base_row'
    ] if c in keep_cols]

    for c in num_cols:
        pairs[c] = pd.to_numeric(pairs[c], errors='coerce')
    for c in cat_cols:
        pairs[c] = pairs[c].astype('category')
    return pairs[num_cols + cat_cols]

In [53]:
test_df = test_to_X(student_df, l2_uni)

In [54]:
test_df

student_diem_chuan,student_budget_max,cand_diem_chuan_final,cand_hoc_phi,cand_y_base,diff_hk10,diff_hk11,diff_hk12,diff_hl10,diff_hl11,diff_hl12,student_cong_lap,student_tinh_tp,student_to_hop_mon,student_ten_ccta,student_diem_ccta,student_nhom_nganh,cand_cong_lap,cand_tinh_tp,cand_to_hop_mon,cand_nhom_nganh,cand_ma_xet_tuyen,cand_is_base_row
f64,i64,f64,i64,f64,i64,i64,i64,i64,i64,f64,i64,cat,cat,cat,cat,f64,i64,cat,cat,i64,cat,cat
800.0,40000000,721.0,13500000,721.0,8,8,8,8,8,8.0,1,"""TP. Hồ Chí Minh""","""VNUHCM""","""0""","""0""",714.0,1,"""TP. Hồ Chí Minh""","""VNUHCM""",714,"""NLS7140215ĐGNL""","""True"""
24.0,40000000,21.0,25000000,21.0,8,8,8,8,8,8.0,1,"""TP. Hồ Chí Minh""","""A00""","""0""","""0""",748.0,1,"""TP. Hồ Chí Minh""","""A00""",748,"""MBS7480201THPTQG""","""True"""
24.0,40000000,22.0,33500000,22.0,8,8,8,8,8,8.0,1,"""TP. Hồ Chí Minh""","""A00""","""0""","""0""",748.0,1,"""TP. Hồ Chí Minh""","""A00""",748,"""IUH7480108CTHPTQG""","""True"""
24.0,40000000,18.0,29400000,18.0,8,8,8,8,8,8.0,1,"""TP. Hồ Chí Minh""","""A00""","""0""","""0""",748.0,1,"""TP. Hồ Chí Minh""","""A00""",748,"""GTS748020105ATHPTQG""","""True"""
24.0,40000000,24.0,33500000,24.0,8,8,8,8,8,8.0,1,"""TP. Hồ Chí Minh""","""A00""","""0""","""0""",748.0,1,"""TP. Hồ Chí Minh""","""A00""",748,"""IUH7480108THPTQG""","""True"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
21.0,40000000,21.25,15500000,21.25,8,8,8,8,8,8.0,1,"""TP. Hồ Chí Minh""","""A01""","""0""","""0""",748.0,1,"""TP. Hồ Chí Minh""","""A01""",748,"""NLS7480104THPTQG""","""True"""
21.0,40000000,25.4,35000000,25.4,8,8,8,8,8,8.0,1,"""TP. Hồ Chí Minh""","""A01""","""0""","""0""",748.0,1,"""TP. Hồ Chí Minh""","""A01""",748,"""KSA7480201THPTQG""","""True"""
21.0,40000000,22.25,15500000,22.25,8,8,8,8,8,8.0,1,"""TP. Hồ Chí Minh""","""A01""","""0""","""0""",748.0,1,"""TP. Hồ Chí Minh""","""A01""",748,"""NLS7480201THPTQG""","""True"""


In [73]:
# test_df = test_df.to_pandas()
test_df.info()

# force test_df to dtype column as X_valid
for c in num_cols:
    test_df[c] = pd.to_numeric(test_df[c], errors='coerce')
for c in cat_cols:
    test_df[c] = test_df[c].astype('category')  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   student_diem_chuan     103 non-null    float64 
 1   student_budget_max     103 non-null    int64   
 2   cand_diem_chuan_final  103 non-null    float64 
 3   cand_hoc_phi           103 non-null    int64   
 4   cand_y_base            103 non-null    float64 
 5   diff_hk10              103 non-null    int64   
 6   diff_hk11              103 non-null    int64   
 7   diff_hk12              103 non-null    int64   
 8   diff_hl10              103 non-null    int64   
 9   diff_hl11              103 non-null    int64   
 10  diff_hl12              103 non-null    float64 
 11  student_cong_lap       103 non-null    int64   
 12  student_tinh_tp        103 non-null    category
 13  student_to_hop_mon     103 non-null    category
 14  student_ten_ccta       103 non-null    cat

In [71]:
X_valid.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49104 entries, 20 to 241335
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   student_diem_chuan     49104 non-null  float64 
 1   student_budget_max     49104 non-null  int64   
 2   cand_diem_chuan_final  49104 non-null  float64 
 3   cand_hoc_phi           49104 non-null  int64   
 4   cand_y_base            49104 non-null  float64 
 5   diff_hk10              49104 non-null  int64   
 6   diff_hk11              49104 non-null  int64   
 7   diff_hk12              49104 non-null  int64   
 8   diff_hl10              49104 non-null  int64   
 9   diff_hl11              49104 non-null  int64   
 10  diff_hl12              49104 non-null  int64   
 11  student_cong_lap       49104 non-null  category
 12  student_tinh_tp        49104 non-null  category
 13  student_to_hop_mon     49104 non-null  category
 14  student_ten_ccta       49104 non-null  ca

In [76]:
p = clf.predict(test_df, num_iteration=clf.best_iteration)
print(p>0.5)

# trả ra những rows trong test_df có p>0.5
test_df[p>0.5]

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True False False False False False False False False False
 False False False False  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True False False False False
 False False False False False False False False False False False False
 False False False False False False False]


,student_diem_chuan,student_budget_max,cand_diem_chuan_final,cand_hoc_phi,cand_y_base,diff_hk10,diff_hk11,diff_hk12,diff_hl10,diff_hl11,...,student_to_hop_mon,student_ten_ccta,student_diem_ccta,student_nhom_nganh,cand_cong_lap,cand_tinh_tp,cand_to_hop_mon,cand_nhom_nganh,cand_ma_xet_tuyen,cand_is_base_row
0,800.0,40000000,721.0,13500000,721.0,8,8,8,8,8,...,VNUHCM,0,0,714.0,1,TP. Hồ Chí Minh,VNUHCM,714,NLS7140215ĐGNL,True
1,24.0,40000000,21.0,25000000,21.0,8,8,8,8,8,...,A00,0,0,748.0,1,TP. Hồ Chí Minh,A00,748,MBS7480201THPTQG,True
2,24.0,40000000,22.0,33500000,22.0,8,8,8,8,8,...,A00,0,0,748.0,1,TP. Hồ Chí Minh,A00,748,IUH7480108CTHPTQG,True
3,24.0,40000000,18.0,29400000,18.0,8,8,8,8,8,...,A00,0,0,748.0,1,TP. Hồ Chí Minh,A00,748,GTS748020105ATHPTQG,True
4,24.0,40000000,24.0,33500000,24.0,8,8,8,8,8,...,A00,0,0,748.0,1,TP. Hồ Chí Minh,A00,748,IUH7480108THPTQG,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,21.0,40000000,17.5,16400000,17.5,8,8,8,8,8,...,A01,0,0,748.0,1,TP. Hồ Chí Minh,A01,748,DTM7480201THPTQG,True
76,21.0,40000000,13.0,19500000,15.0,8,8,8,8,8,...,A01,0,0,748.0,1,TP. Hồ Chí Minh,A01,748,DSG7480201THPTQG,False
77,21.0,40000000,18.0,13200000,18.0,8,8,8,8,8,...,A01,0,0,748.0,1,TP. Hồ Chí Minh,A01,748,HHK7480201THPTQG,True
78,21.0,40000000,18.0,29400000,18.0,8,8,8,8,8,...,A01,0,0,748.0,1,TP. Hồ Chí Minh,A01,748,GTS748020105ATHPTQG,True


In [81]:
import os, json

model_path = "../../../models/user_item_lightgbm/l2_lightgbm.txt"  # nên dùng .txt cho Booster
os.makedirs(os.path.dirname(model_path), exist_ok=True)

clf.save_model(model_path, num_iteration=getattr(clf, "best_iteration", clf.current_iteration()))

with open("../../../models/user_item_lightgbm/feature_names.json","w",encoding="utf-8") as f:
    json.dump(clf.feature_name(), f, ensure_ascii=False)
with open("../../../models/user_item_lightgbm/cat_vocab.json","w",encoding="utf-8") as f:
    json.dump({c: [str(v) for v in X_train[c].cat.categories] for c in X_train.columns if str(X_train[c].dtype)=="category"}, f, ensure_ascii=False)

print(f"Saved model to {model_path}")


Saved model to ../../../models/user_item_lightgbm/l2_lightgbm.txt
